과소적합(underfitting)
- 모델이 너무 단순하거나, 규제가 너무 많거나, 충분한 훈련을 하지 않은 경우 발생
- 네트워크가 훈련 세트에서 적절한 패턴을 학습하지 못했다는 의미
- 그러나 너무 오래 훈련하면 모델이 과대적합(overfitting)되어 테스트 데이터에 일반화되지 않는 훈련 데이터로부터 패턴을 학습하게 된다.

1. 적절한 epoch 수로 훈련
2. 완전한 훈련 데이터 사용: 모델이 처리할 것으로 예상되는 전체 입력 범위를 포괄하는 것이 좋다.
3. 정규화: 모델이 저장할 수 있는 정보의 양과 유형에 제약을 가하여, 적은 수의 패턴만 기억하게 만들어 일반화 가능성이 더 높은 패턴을 중점으로 최적화하게 한다.


In [1]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

2.16.1


In [8]:
%pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

Note: you may need to restart the kernel to use updated packages.Collecting git+https://github.com/tensorflow/docs
  Cloning https://github.com/tensorflow/docs to c:\users\서민지\appdata\local\temp\pip-req-build-fcgco9fj
  Resolved https://github.com/tensorflow/docs to commit ec3a1b3f9a604c6cd26c7537f2b9fd270a3eedbf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
   ---------------------------------------- 0.0/133.2 kB ? eta -:--:--
   ------------------------ --------------- 81.9/133.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 133.2/133.2 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.4 kB ? eta -:--:--
   ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   -----------

  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\서민지\AppData\Local\Temp\pip-req-build-fcgco9fj'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

In [10]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [11]:
# 힉스(Higgs) 데이터세트, 28개의 특성과, 이진 클래스 레이블이 있는 11,000,000개

gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

  23199744/2816407858 ━━━━━━━━━━━━━━━━━━━━ 29:34:55 38us/step

힉스 데이터 세트
- 28개의 특성(Features)
- 이진 클래스 레이블(Binary Class Labels): 데이터 포인트에 대한 목표 변수
- 데이터 구조: 11,000,000 x 28 크기의 행렬 (데이터 포인트 x 특성)
- 대규모 데이터셋의 경우 전처리 과정이 필요할 수있다.
    - 데이터의 결측치 처리
    - 이상치 제거
    - 특성 스케일링
    - 범주형 변수의 인코딩

In [ ]:
FEATURES = 28

In [ ]:
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURES+1), compression_type="GZIP")

In [ ]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label  # feature_vector, label 쌍으로 다시 압축

In [ ]:
# 대규모 데이터 배치에서 작업
packed_ds = ds.batch(10000).map(pack_row).unbatch()

In [ ]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

In [ ]:
N_VALIDATION = int(1e3) # 1000
N_TRAIN = int(1e4)      # 10000
BUFFER_SIZE = int(1e4)  # 데이터셋을 셔플링할 때 사용되는 버퍼의 크기
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

In [ ]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
# 로더가 각 epoch 에서 파일 데이터를 다시 로딩할 필요 없도록 캐싱
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [ ]:
train_ds